In [1]:
import sqlite3
import bcrypt

# Connect to SQLite database
conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()

# Create Admin Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Admin (
    admin_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL
)''')

# Create Doctor Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Doctor (
    doctor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    specialization TEXT NOT NULL
)''')

# Create Staff Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Staff (
    staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    designation TEXT NOT NULL
)''')

# Create Patient Table (Includes In-Patient & Out-Patient)
cursor.execute('''CREATE TABLE IF NOT EXISTS Patient (
    patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    gender TEXT NOT NULL,
    admission_type TEXT CHECK(admission_type IN ('In-Patient', 'Out-Patient'))
)''')

# Create Room Table (For In-Patients)
cursor.execute('''CREATE TABLE IF NOT EXISTS Room (
    room_id INTEGER PRIMARY KEY AUTOINCREMENT,
    room_no TEXT NOT NULL UNIQUE,
    room_cost REAL NOT NULL,
    patient_id INTEGER,
    FOREIGN KEY (patient_id) REFERENCES Patient(patient_id)
)''')

# Create Medical Info Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Medical_Info (
    record_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER NOT NULL,
    doctor_id INTEGER NOT NULL,
    medicine TEXT NOT NULL,
    FOREIGN KEY (patient_id) REFERENCES Patient(patient_id),
    FOREIGN KEY (doctor_id) REFERENCES Doctor(doctor_id)
)''')

# Create Billing Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Billing (
    bill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER NOT NULL,
    total REAL NOT NULL,
    FOREIGN KEY (patient_id) REFERENCES Patient(patient_id)
)''')

conn.commit()
conn.close()

print("Database and tables created successfully!")


Database and tables created successfully!


In [2]:
def register_admin(username, password):
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    
    try:
        cursor.execute("INSERT INTO Admin (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        print("Admin registered successfully!")
    except sqlite3.IntegrityError:
        print("Username already exists.")
    
    conn.close()


In [3]:
def login(username, password):
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT password FROM Admin WHERE username=?", (username,))
    result = cursor.fetchone()
    
    if result and bcrypt.checkpw(password.encode('utf-8'), result[0]):
        print("Login Successful!")
    else:
        print("Invalid username or password.")
    
    conn.close()


In [4]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import bcrypt

# Function to authenticate login
def login():
    username = entry_username.get()
    password = entry_password.get()

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()

    # Check in Admin table
    cursor.execute("SELECT password FROM Admin WHERE username=?", (username,))
    result = cursor.fetchone()
    
    if result and bcrypt.checkpw(password.encode('utf-8'), result[0]):
        messagebox.showinfo("Login Successful", f"Welcome, {username} (Admin)")
        root.destroy()  # Close login window
        open_dashboard("Admin")  # Open Admin Dashboard
        return
    
    # Check in Doctor table
    cursor.execute("SELECT doctor_id FROM Doctor WHERE name=?", (username,))
    result = cursor.fetchone()
    if result:
        messagebox.showinfo("Login Successful", f"Welcome, Dr. {username}")
        root.destroy()
        open_dashboard("Doctor")
        return

    # Check in Staff table
    cursor.execute("SELECT staff_id, designation FROM Staff WHERE name=?", (username,))
    result = cursor.fetchone()
    if result:
        messagebox.showinfo("Login Successful", f"Welcome, {username} ({result[1]})")
        root.destroy()
        open_dashboard(result[1])
        return

    messagebox.showerror("Login Failed", "Invalid username or password")
    conn.close()

# Function to open respective dashboards
def open_dashboard(role):
    dashboard = tk.Tk()
    dashboard.title(f"{role} Dashboard")
    dashboard.geometry("400x300")
    
    label = tk.Label(dashboard, text=f"Welcome to {role} Dashboard", font=("Arial", 14))
    label.pack(pady=20)
    
    dashboard.mainloop()

# Tkinter Login Window
root = tk.Tk()
root.title("Hospital Management Login")
root.geometry("400x300")

tk.Label(root, text="Username:", font=("Arial", 12)).pack(pady=5)
entry_username = tk.Entry(root, font=("Arial", 12))
entry_username.pack(pady=5)

tk.Label(root, text="Password:", font=("Arial", 12)).pack(pady=5)
entry_password = tk.Entry(root, font=("Arial", 12), show="*")
entry_password.pack(pady=5)

login_btn = tk.Button(root, text="Login", font=("Arial", 12), command=login)
login_btn.pack(pady=20)

root.mainloop()


2025-04-23 11:54:40.369 Python[2295:75662] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-23 11:54:40.369 Python[2295:75662] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [5]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import matplotlib.pyplot as plt

# Function to fetch statistics
def fetch_stats():
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT COUNT(*) FROM Doctor")
    total_doctors = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM Staff")
    total_staff = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Patient")
    total_patients = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Room")
    total_rooms = cursor.fetchone()[0]

    conn.close()
    
    return total_doctors, total_staff, total_patients, total_rooms

# Function to display statistics
def show_stats():
    doctors, staff, patients, rooms = fetch_stats()
    
    labels = ['Doctors', 'Staff', 'Patients', 'Rooms']
    values = [doctors, staff, patients, rooms]

    plt.figure(figsize=(6, 4))
    plt.bar(labels, values, color=['blue', 'green', 'red', 'purple'])
    plt.xlabel("Categories")
    plt.ylabel("Count")
    plt.title("Hospital Overview")
    plt.show()

# Function to open doctor management window
def manage_doctors():
    messagebox.showinfo("Doctors", "Manage Doctors (Feature Coming Soon)")

# Function to open staff management window
def manage_staff():
    messagebox.showinfo("Staff", "Manage Staff (Feature Coming Soon)")

# Function to open patient management window
def manage_patients():
    messagebox.showinfo("Patients", "Manage Patients (Feature Coming Soon)")

# Function to open billing management window
def manage_billing():
    messagebox.showinfo("Billing", "Manage Billing (Feature Coming Soon)")

# Function to log out
def logout():
    admin_dashboard.destroy()
    import login  # Redirect back to login screen

# Create Admin Dashboard
admin_dashboard = tk.Tk()
admin_dashboard.title("Admin Dashboard")
admin_dashboard.geometry("600x400")

tk.Label(admin_dashboard, text="Hospital Management System - Admin", font=("Arial", 16, "bold")).pack(pady=10)

# Buttons
btn_doctor = tk.Button(admin_dashboard, text="Manage Doctors", font=("Arial", 12), command=manage_doctors)
btn_doctor.pack(pady=5)

btn_staff = tk.Button(admin_dashboard, text="Manage Staff", font=("Arial", 12), command=manage_staff)
btn_staff.pack(pady=5)

btn_patients = tk.Button(admin_dashboard, text="Manage Patients", font=("Arial", 12), command=manage_patients)
btn_patients.pack(pady=5)

btn_billing = tk.Button(admin_dashboard, text="Manage Billing", font=("Arial", 12), command=manage_billing)
btn_billing.pack(pady=5)

btn_stats = tk.Button(admin_dashboard, text="View Hospital Statistics", font=("Arial", 12), command=show_stats)
btn_stats.pack(pady=5)

btn_logout = tk.Button(admin_dashboard, text="Logout", font=("Arial", 12), command=logout, fg="red")
btn_logout.pack(pady=20)

admin_dashboard.mainloop()


In [6]:

conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()

# Hash the password securely
password = "admin123"
hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

# Insert Admin account
try:
    cursor.execute("INSERT INTO Admin (username, password) VALUES (?, ?)", ("admin", hashed_password))
    conn.commit()
    print("Admin account created successfully! Username: admin, Password: admin123")
except sqlite3.IntegrityError:
    print("Admin account already exists!")

conn.close()


Admin account already exists!


In [7]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# Function to refresh the doctor list
def refresh_doctors():
    for row in tree.get_children():
        tree.delete(row)
    
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Doctor")
    doctors = cursor.fetchall()
    conn.close()

    for doctor in doctors:
        tree.insert("", "end", values=doctor)

# Function to add a doctor
def add_doctor():
    name = entry_name.get()
    specialization = entry_specialization.get()

    if name and specialization:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Doctor (name, specialization) VALUES (?, ?)", (name, specialization))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Doctor added successfully!")
        entry_name.delete(0, tk.END)
        entry_specialization.delete(0, tk.END)
        refresh_doctors()
    else:
        messagebox.showerror("Error", "All fields are required!")

# Function to delete a selected doctor
def delete_doctor():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No doctor selected!")
        return
    
    doctor_id = tree.item(selected_item)["values"][0]
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM Doctor WHERE doctor_id=?", (doctor_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Doctor deleted successfully!")
    refresh_doctors()

# Create Tkinter Window
root = tk.Tk()
root.title("Doctor Management")
root.geometry("600x400")

# Labels & Entry Fields
tk.Label(root, text="Doctor Name:", font=("Arial", 12)).pack(pady=5)
entry_name = tk.Entry(root, font=("Arial", 12))
entry_name.pack(pady=5)

tk.Label(root, text="Specialization:", font=("Arial", 12)).pack(pady=5)
entry_specialization = tk.Entry(root, font=("Arial", 12))
entry_specialization.pack(pady=5)

# Buttons
btn_add = tk.Button(root, text="Add Doctor", font=("Arial", 12), command=add_doctor)
btn_add.pack(pady=5)

btn_delete = tk.Button(root, text="Delete Selected Doctor", font=("Arial", 12), command=delete_doctor, fg="red")
btn_delete.pack(pady=5)

# Doctor List (TreeView)
columns = ("Doctor ID", "Name", "Specialization")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("Doctor ID", text="Doctor ID")
tree.heading("Name", text="Name")
tree.heading("Specialization", text="Specialization")
tree.pack(pady=10)

# Load existing doctors
refresh_doctors()

root.mainloop()


In [8]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# Function to refresh the staff list
def refresh_staff():
    for row in tree.get_children():
        tree.delete(row)

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Staff")
    staff_members = cursor.fetchall()
    conn.close()

    for staff in staff_members:
        tree.insert("", "end", values=staff)

# Function to add a staff member
def add_staff():
    name = entry_name.get()
    designation = entry_designation.get()

    if name and designation:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Staff (name, designation) VALUES (?, ?)", (name, designation))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Staff member added successfully!")
        entry_name.delete(0, tk.END)
        entry_designation.delete(0, tk.END)
        refresh_staff()
    else:
        messagebox.showerror("Error", "All fields are required!")

# Function to delete a selected staff member
def delete_staff():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No staff member selected!")
        return

    staff_id = tree.item(selected_item)["values"][0]
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM Staff WHERE staff_id=?", (staff_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Staff member deleted successfully!")
    refresh_staff()

# Create Tkinter Window
root = tk.Tk()
root.title("Staff Management")
root.geometry("600x400")

# Labels & Entry Fields
tk.Label(root, text="Staff Name:", font=("Arial", 12)).pack(pady=5)
entry_name = tk.Entry(root, font=("Arial", 12))
entry_name.pack(pady=5)

tk.Label(root, text="Designation:", font=("Arial", 12)).pack(pady=5)
entry_designation = tk.Entry(root, font=("Arial", 12))
entry_designation.pack(pady=5)

# Buttons
btn_add = tk.Button(root, text="Add Staff", font=("Arial", 12), command=add_staff)
btn_add.pack(pady=5)

btn_delete = tk.Button(root, text="Delete Selected Staff", font=("Arial", 12), command=delete_staff, fg="red")
btn_delete.pack(pady=5)

# Staff List (TreeView)
columns = ("Staff ID", "Name", "Designation")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("Staff ID", text="Staff ID")
tree.heading("Name", text="Name")
tree.heading("Designation", text="Designation")
tree.pack(pady=10)

# Load existing staff members
refresh_staff()

root.mainloop()


In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# Function to refresh the patient list
def refresh_patients():
    for row in tree.get_children():
        tree.delete(row)

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Patient")
    patients = cursor.fetchall()
    conn.close()

    for patient in patients:
        tree.insert("", "end", values=patient)

# Function to add a new patient
def add_patient():
    name = entry_name.get()
    gender = gender_var.get()
    admission_type = admission_var.get()
    assigned_doctor = entry_doctor.get()

    if name and gender and admission_type and assigned_doctor:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Patient (name, gender, admission_type) VALUES (?, ?, ?)", 
                       (name, gender, admission_type))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Patient registered successfully!")
        entry_name.delete(0, tk.END)
        entry_doctor.delete(0, tk.END)
        refresh_patients()
    else:
        messagebox.showerror("Error", "All fields are required!")

# Function to delete a selected patient
def delete_patient():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No patient selected!")
        return

    patient_id = tree.item(selected_item)["values"][0]
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM Patient WHERE patient_id=?", (patient_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Patient deleted successfully!")
    refresh_patients()

# Create Tkinter Window
root = tk.Tk()
root.title("Patient Management")
root.geometry("700x500")

# Labels & Entry Fields
tk.Label(root, text="Patient Name:", font=("Arial", 12)).pack(pady=5)
entry_name = tk.Entry(root, font=("Arial", 12))
entry_name.pack(pady=5)

tk.Label(root, text="Gender:", font=("Arial", 12)).pack(pady=5)
gender_var = tk.StringVar(value="Male")
tk.Radiobutton(root, text="Male", variable=gender_var, value="Male").pack()
tk.Radiobutton(root, text="Female", variable=gender_var, value="Female").pack()

tk.Label(root, text="Admission Type:", font=("Arial", 12)).pack(pady=5)
admission_var = tk.StringVar(value="In-Patient")
tk.Radiobutton(root, text="In-Patient", variable=admission_var, value="In-Patient").pack()
tk.Radiobutton(root, text="Out-Patient", variable=admission_var, value="Out-Patient").pack()

tk.Label(root, text="Assigned Doctor:", font=("Arial", 12)).pack(pady=5)
entry_doctor = tk.Entry(root, font=("Arial", 12))
entry_doctor.pack(pady=5)

# Buttons
btn_add = tk.Button(root, text="Register Patient", font=("Arial", 12), command=add_patient)
btn_add.pack(pady=5)

btn_delete = tk.Button(root, text="Delete Selected Patient", font=("Arial", 12), command=delete_patient, fg="red")
btn_delete.pack(pady=5)

# Patient List (TreeView)
columns = ("Patient ID", "Name", "Gender", "Admission Type")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("Patient ID", text="Patient ID")
tree.heading("Name", text="Name")
tree.heading("Gender", text="Gender")
tree.heading("Admission Type", text="Admission Type")
tree.pack(pady=10)

# Load existing patients
refresh_patients()

root.mainloop()


In [10]:
import sqlite3

conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()

# Create Room Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Room (
    room_id INTEGER PRIMARY KEY AUTOINCREMENT,
    room_no TEXT UNIQUE NOT NULL,
    room_cost REAL NOT NULL,
    patient_id INTEGER NULL,
    FOREIGN KEY (patient_id) REFERENCES Patient(patient_id)
)''')

conn.commit()
conn.close()
print("Room table created successfully!")


Room table created successfully!


In [11]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# Function to refresh the room list
def refresh_rooms():
    for row in tree.get_children():
        tree.delete(row)

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Room")
    rooms = cursor.fetchall()
    conn.close()

    for room in rooms:
        tree.insert("", "end", values=room)

# Function to add a new room
def add_room():
    room_no = entry_room_no.get()
    room_cost = entry_room_cost.get()

    if room_no and room_cost:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()
        try:
            cursor.execute("INSERT INTO Room (room_no, room_cost) VALUES (?, ?)", (room_no, room_cost))
            conn.commit()
            messagebox.showinfo("Success", "Room added successfully!")
            entry_room_no.delete(0, tk.END)
            entry_room_cost.delete(0, tk.END)
            refresh_rooms()
        except sqlite3.IntegrityError:
            messagebox.showerror("Error", "Room number already exists!")
        conn.close()
    else:
        messagebox.showerror("Error", "All fields are required!")

# Function to delete a selected room
def delete_room():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No room selected!")
        return

    room_id = tree.item(selected_item)["values"][0]
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM Room WHERE room_id=?", (room_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Room deleted successfully!")
    refresh_rooms()

# Function to assign a room to a patient
def assign_room():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No room selected!")
        return

    room_id = tree.item(selected_item)["values"][0]
    patient_id = entry_patient_id.get()

    if patient_id:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()

        # Check if the patient exists
        cursor.execute("SELECT patient_id FROM Patient WHERE patient_id=?", (patient_id,))
        patient = cursor.fetchone()
        
        if patient:
            cursor.execute("UPDATE Room SET patient_id=? WHERE room_id=?", (patient_id, room_id))
            conn.commit()
            conn.close()
            messagebox.showinfo("Success", "Room assigned to patient successfully!")
            refresh_rooms()
        else:
            messagebox.showerror("Error", "Invalid Patient ID!")
            conn.close()
    else:
        messagebox.showerror("Error", "Enter Patient ID!")

# Create Tkinter Window
root = tk.Tk()
root.title("Room Management")
root.geometry("700x500")

# Labels & Entry Fields
tk.Label(root, text="Room No:", font=("Arial", 12)).pack(pady=5)
entry_room_no = tk.Entry(root, font=("Arial", 12))
entry_room_no.pack(pady=5)

tk.Label(root, text="Room Cost:", font=("Arial", 12)).pack(pady=5)
entry_room_cost = tk.Entry(root, font=("Arial", 12))
entry_room_cost.pack(pady=5)

# Buttons
btn_add = tk.Button(root, text="Add Room", font=("Arial", 12), command=add_room)
btn_add.pack(pady=5)

btn_delete = tk.Button(root, text="Delete Selected Room", font=("Arial", 12), command=delete_room, fg="red")
btn_delete.pack(pady=5)

# Assign Room Section
tk.Label(root, text="Assign Room to Patient (Enter Patient ID):", font=("Arial", 12)).pack(pady=5)
entry_patient_id = tk.Entry(root, font=("Arial", 12))
entry_patient_id.pack(pady=5)

btn_assign = tk.Button(root, text="Assign Room", font=("Arial", 12), command=assign_room, fg="blue")
btn_assign.pack(pady=5)

# Room List (TreeView)
columns = ("Room ID", "Room No", "Room Cost", "Assigned Patient ID")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("Room ID", text="Room ID")
tree.heading("Room No", text="Room No")
tree.heading("Room Cost", text="Room Cost")
tree.heading("Assigned Patient ID", text="Assigned Patient ID")
tree.pack(pady=10)

# Load existing rooms
refresh_rooms()

root.mainloop()


In [12]:
import sqlite3

conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()

# Create Billing Table
cursor.execute('''CREATE TABLE IF NOT EXISTS Billing (
    bill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER NOT NULL,
    total REAL NOT NULL,
    status TEXT CHECK(status IN ('Paid', 'Pending')),
    FOREIGN KEY (patient_id) REFERENCES Patient(patient_id)
)''')

conn.commit()
conn.close()
print("Billing table created successfully!")


Billing table created successfully!


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from reportlab.pdfgen import canvas

# Function to refresh the bill list
def refresh_bills():
    for row in tree.get_children():
        tree.delete(row)

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Billing")
    bills = cursor.fetchall()
    conn.close()

    for bill in bills:
        tree.insert("", "end", values=bill)

# Function to generate a new bill
def add_bill():
    patient_id = entry_patient_id.get()
    total = entry_total.get()
    status = status_var.get()

    if patient_id and total and status:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()
        
        # Validate patient ID
        cursor.execute("SELECT patient_id FROM Patient WHERE patient_id=?", (patient_id,))
        patient = cursor.fetchone()
        if not patient:
            messagebox.showerror("Error", "Invalid Patient ID!")
            conn.close()
            return

        cursor.execute("INSERT INTO Billing (patient_id, total, status) VALUES (?, ?, ?)", 
                       (patient_id, total, status))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Bill generated successfully!")
        entry_patient_id.delete(0, tk.END)
        entry_total.delete(0, tk.END)
        refresh_bills()
    else:
        messagebox.showerror("Error", "All fields are required!")

# Function to delete a selected bill
def delete_bill():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No bill selected!")
        return

    bill_id = tree.item(selected_item)["values"][0]
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM Billing WHERE bill_id=?", (bill_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Bill deleted successfully!")
    refresh_bills()

# Function to export a bill as PDF
def export_bill():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No bill selected!")
        return

    bill_details = tree.item(selected_item)["values"]
    bill_id, patient_id, total, status = bill_details

    pdf_filename = f"Bill_{bill_id}.pdf"
    c = canvas.Canvas(pdf_filename)
    c.setFont("Helvetica", 12)

    c.drawString(100, 750, f"Hospital Management System - Bill")
    c.drawString(100, 730, f"Bill ID: {bill_id}")
    c.drawString(100, 710, f"Patient ID: {patient_id}")
    c.drawString(100, 690, f"Total Amount: ₹{total}")
    c.drawString(100, 670, f"Payment Status: {status}")

    c.save()
    messagebox.showinfo("Success", f"Bill exported as {pdf_filename}!")

# Create Tkinter Window
root = tk.Tk()
root.title("Billing Management")
root.geometry("700x500")

# Labels & Entry Fields
tk.Label(root, text="Patient ID:", font=("Arial", 12)).pack(pady=5)
entry_patient_id = tk.Entry(root, font=("Arial", 12))
entry_patient_id.pack(pady=5)

tk.Label(root, text="Total Amount (₹):", font=("Arial", 12)).pack(pady=5)
entry_total = tk.Entry(root, font=("Arial", 12))
entry_total.pack(pady=5)

tk.Label(root, text="Payment Status:", font=("Arial", 12)).pack(pady=5)
status_var = tk.StringVar(value="Pending")
status_dropdown = ttk.Combobox(root, textvariable=status_var, values=["Paid", "Pending"])
status_dropdown.pack(pady=5)

# Buttons
btn_add = tk.Button(root, text="Generate Bill", font=("Arial", 12), command=add_bill)
btn_add.pack(pady=5)

btn_delete = tk.Button(root, text="Delete Selected Bill", font=("Arial", 12), command=delete_bill, fg="red")
btn_delete.pack(pady=5)

btn_export = tk.Button(root, text="Export Bill as PDF", font=("Arial", 12), command=export_bill, fg="blue")
btn_export.pack(pady=5)

# Billing List (TreeView)
columns = ("Bill ID", "Patient ID", "Total Amount", "Payment Status")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("Bill ID", text="Bill ID")
tree.heading("Patient ID", text="Patient ID")
tree.heading("Total Amount", text="Total Amount")
tree.heading("Payment Status", text="Payment Status")
tree.pack(pady=10)

# Load existing bills
refresh_bills()

root.mainloop()


: 

In [ ]:

num = int(input("Enter a number: "))
num_str = str(num)
num_digits = len(num_str)

armstrong_sum = 0
for digit in num_str:
    armstrong_sum += int(digit) ** num_digits


if armstrong_sum == num:
    print(f"{num} is an Armstrong number.")
else:
    print(f"{num} is not an Armstrong number.")
